<a href="https://colab.research.google.com/github/vijjus/CV/blob/master/CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [2]:
!echo '3FDsALdx4sLyMedGxO5h:RsdZX1E2jRryINc6Vb5HGpL1FY84lNBXbTNSU8Ln' > ~/.passwd-s3fs
!cat ~/.passwd-s3fs
!chmod 600 ~/.passwd-s3fs
!mkdir /s3
!apt install s3fs

3FDsALdx4sLyMedGxO5h:RsdZX1E2jRryINc6Vb5HGpL1FY84lNBXbTNSU8Ln
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  s3fs
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 200 kB of archives.
After this operation, 557 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 s3fs amd64 1.82-1 [200 kB]
Fetched 200 kB in 0s (1,246 kB/s)
Selecting previously unselected package s3fs.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../archives/s3fs_1.82-1_amd64.deb ...
Unpacking s3fs (1.82-1) ...
Setting up s3fs (1.82-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [3]:
!s3fs w251-final-project /s3 -o passwd_file=~/.passwd-s3fs,url=https://s3.us.cloud-object-storage.appdomain.cloud
!ls /s3

 caption_model.pth.tar
 CatImages
 dataset_coco.json.gz
 delivery.jpg
 DFDC
 haderfake.mp4
 image2.jpeg
 image3.jpeg
 image_captioning.ipynb
 kagglecatsanddogs_3367a.zip
 kat_ring0.mp4
 kat_ring1.mp4
 kat_ring2.mp4
 kat_ring3.mp4
 knowns
 model.pt.zip
 models.py
'MSR-LA - 3467.docx'
 new_faces.ipynb
 package.jpg
 package.mp4
 PetImages
 pman.jpg
 pretrained_weights
'readme[1].txt'
 Reuters_Deepfake.mp4
 testdir
 test_imgs
 uploads
 WORDMAP_coco_5_cap_per_img_5_min_word_freq.json.gz


In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt

In [12]:
!pip3 install timm

     |████████████████████████████████| 184kB 7.8MB/s 


In [0]:
import timm

In [5]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [0]:
input_size = 512
output_size = 1
hidden_dim = 512
n_layers = 2

In [0]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [0]:
class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        # use mixnet_s, remove the final Linear layer (1536 -> 1000)
        self.mixnet = timm.create_model("mixnet_s", pretrained=True)
        self.mixnet.classifier = Identity()

        self.lstm = nn.LSTM(1536, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_dim, 32)
        self.fc4 = nn.Linear(32, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size, seqlen, c, h, w = x.size()
        x = x.reshape(batch_size*seqlen, c, h, w).float()
        x = self.mixnet(x)
        x = x.reshape(batch_size, seqlen, x.shape[1])
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc1(out)
        out = self.elu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [0]:
net = DFDCNet(input_size, output_size, hidden_dim, n_layers)

In [0]:
m = timm.create_model("mixnet_s", pretrained=True)

In [0]:
m.classifier = Identity()

In [0]:
lstm = nn.LSTM(1536, 512, 2, dropout=0.5, batch_first=True)

In [0]:
x = torch.randn(16, 2, 3, 224, 224)

In [23]:
x.size()

torch.Size([16, 2, 3, 224, 224])

In [0]:
batch_size, seqlen, c, h, w = x.size()

In [0]:
x = x.reshape(batch_size*seqlen, c, h, w).float()

In [26]:
x.shape

torch.Size([32, 3, 224, 224])

In [0]:
# run it through the CNN first
x = m(x)

In [28]:
x.shape

torch.Size([32, 1536])

In [0]:
x = x.reshape(batch_size, seqlen, x.shape[1])

In [30]:
x.shape

torch.Size([16, 2, 1536])

In [0]:
hidden = net.init_hidden(16)

In [38]:
hidden[0].shape, hidden[1].shape

(torch.Size([2, 16, 512]), torch.Size([2, 16, 512]))

In [0]:
lstm_out, hidden = lstm(x, hidden)

In [45]:
type(lstm_out)

torch.Tensor

In [46]:
lstm_out.shape

torch.Size([16, 2, 512])

In [41]:
hidden[0]

tensor([[[ 0.0861, -0.3216,  0.0747,  ...,  0.0136,  0.0574, -0.1885],
         [ 0.1063, -0.1820, -0.1261,  ..., -0.0233,  0.0555, -0.4650],
         [ 0.1695, -0.1003,  0.1754,  ..., -0.0158,  0.0210, -0.2867],
         ...,
         [ 0.0434, -0.2564, -0.0063,  ...,  0.0630,  0.0907, -0.2971],
         [ 0.0540, -0.0032, -0.0724,  ..., -0.1105, -0.1417, -0.3451],
         [ 0.0643, -0.2997, -0.2645,  ..., -0.0890, -0.0713, -0.1612]],

        [[ 0.0573, -0.0396,  0.0603,  ...,  0.0092,  0.0017,  0.0894],
         [ 0.0754,  0.0141, -0.0305,  ...,  0.0033, -0.0542,  0.0115],
         [ 0.0998, -0.0154,  0.0629,  ...,  0.0304, -0.0604, -0.0026],
         ...,
         [ 0.0353,  0.0136,  0.0147,  ...,  0.0480, -0.0173,  0.0465],
         [ 0.0583,  0.0303, -0.0016,  ...,  0.0313,  0.0188,  0.0261],
         [ 0.0375, -0.0143,  0.0261,  ...,  0.0140, -0.0241,  0.0033]]],
       grad_fn=<StackBackward>)

In [0]:
lstm_out = lstm_out.contiguous().view(-1, hidden_dim)

In [43]:
lstm_out.shape

torch.Size([32, 512])

In [0]:
fc = nn.Linear(512, 1)

In [0]:
out = fc(lstm_out)

In [0]:
sigmoid = nn.Sigmoid()

In [0]:
out = sigmoid(out)

In [52]:
out

tensor([[[0.4890],
         [0.4891]],

        [[0.4953],
         [0.4995]],

        [[0.4992],
         [0.4976]],

        [[0.4938],
         [0.4915]],

        [[0.4898],
         [0.4955]],

        [[0.4969],
         [0.4973]],

        [[0.5014],
         [0.4960]],

        [[0.4920],
         [0.4908]],

        [[0.4971],
         [0.4933]],

        [[0.4912],
         [0.4909]],

        [[0.4950],
         [0.4948]],

        [[0.4935],
         [0.4909]],

        [[0.4934],
         [0.4922]],

        [[0.4975],
         [0.4981]],

        [[0.4976],
         [0.4954]],

        [[0.5012],
         [0.5007]]], grad_fn=<SigmoidBackward>)